In [1]:
import sys
sys.path.insert(0, '/VIS_3TH/')

import numpy as np
from numba import njit

from setup import SHORT_PATH
import importlib.util
game_name = 'MachiKoro'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

In [2]:
policy = np.array([
    19, 10, 32, 26, 35, 44, 14, 
    4, 36, 29, 43, 25, 16, 28, 
    2, 37, 33, 41, 9, 21, 11, 
    38, 34, 7, 31, 6, 47, 46, 
    17, 18, 22, 42, 8, 27, 49, 
    54, 45, 24, 39, 50, 3, 5, 
    23, 13, 30, 15, 40, 20, 
    12, 51, 52, 53, 48, 1
])


In [3]:
@njit()
def player(state, per):
    validActions = getValidActions(state)
    validActions[48] = 0
    validActions[52] = state[18]*validActions[52]
    for i in (45, 44, 43, 41, 40, 42, 46, 47):
        if (state[i-33] >= 2):
            validActions[i] = 0
    validActions = np.where(validActions == 1)[0]

    if (0 in validActions) and (np.sum(state[1:16]) <= 10): # Đổ xúc xắc
        return 0, per
    elif (1 in validActions):
        return 1, per
    
    for i in (35, 34, 37, 36, 38, 39):
        if i in validActions:
            return i, per 

    for i in (51, 52, 50, 49, 53):
        if i in validActions:
            return i, per
             
    action = validActions[np.random.randint(len(validActions))]
    return action, per

In [5]:
win, per = numba_main_2(player, 10000, policy, 1)
win

2810